## 💤 [Day 23](https://adventofcode.com/2018/day/23)

In [0]:
import re
import numpy as np

def l1dist(A, B):
  """Manhattan distance"""
  return np.sum(np.abs(A - B), axis=-1)


def get_strongest_range(inputs):
  """Find number of nanobots in range of the bot with strongest range"""
  strongest = np.argmax(inputs[:, -1])
  strongest = inputs[strongest][None, :]
  distances = l1dist(strongest[:, :-1], inputs[:, :-1])
  return np.sum(distances <= strongest[0, -1])

In [2]:
with open('day23.txt', 'r') as f:
  inputs = np.array([[- int(sub[1:]) if sub[0] == '-' else int(sub) 
                      for sub in re.split('<|,|>|=', line) if (sub.isdigit() or sub[1:].isdigit())]
                     for line in f.read().splitlines() if line.strip()])
  
print('Nanobots in range of the strongest:', get_strongest_range(inputs))

Nanobots in range of the strongest: 248


For Part 2, we have to solve an optimization problem with a very large search space.
The objective function is non-convex (L1 norm) so using techniques such as Gradient Descent does not give guarantees. However the problem can be phrases as a **Mixed Integer Linear Programming obejctive**:

\begin{align}
\arg\max_{x, y, z} \sum_{i = 1}^n [\!| \  |x - x_i| + |y - y_i| + |z - z_i| \leq r_i \ |\!]
\end{align}

Expressed as a constraint problem, this becomes:

\begin{align}
\arg\max_{x, y, z}\sum_{i = 1}^n c_i  \mbox{        with }  c_i = 1 \mbox{ if }  |x - x_i| + |y - y_i| + |z - z_i| \leq r_i  \mbox{ else } 0
\end{align}

The constraint can be expressed using only linear terms using the facts that (i) [the conditional on $c_i$ can be expressed with a "dummy" large constant $K$](https://blog.adamfurmanek.pl/blog/2015/09/12/ilp-part-4/) and (ii) absolute values can be turned in 2 constraints (resulting in $8 = 2^3$ constraints for the sum of three absolute values):

\begin{align}
0 \leq c_i \leq 1\\
(x - x_i) + (y - y_i) + (z - z_i) \leq r_i + K (1 - c)\\
(x - x_i) + (y - y_i) - (z - z_i) \leq r_i + K (1 - c)\\
(x - x_i) - (y - y_i) + (z - z_i) \leq r_i + K (1 - c)\\
(x - x_i) - (y - y_i) - (z - z_i) \leq r_i + K (1 - c)\\
- (x - x_i) + (y - y_i) + (z - z_i) \leq r_i + K (1 - c)\\
- (x - x_i) + (y - y_i) - (z - z_i) \leq r_i + K (1 - c)\\
- (x - x_i) - (y - y_i) + (z - z_i) \leq r_i + K (1 - c)\\
- (x - x_i) - (y - y_i) - (z - z_i) \leq r_i + K (1 - c)\\
\end{align}

We can use the [```PulP```](https://pythonhosted.org/PuLP/main/installing_pulp_at_home.html)  library to solve this problem

In [3]:
import pulp

def get_widest_in_range(inputs, K=10000000000):
  """Integer optimization problem using PuLP.
    Using simpler techniques (e.g. gradient descent) does not have strong guarantees since the loss 
    function is non-convex (L1) and risk of falling in local maxima"""
  
  # Variables
  mins = np.amin(inputs[:, :-1], axis=0)
  maxs = np.amax(inputs[:, :-1], axis=0)
  lp_problem = pulp.LpProblem("Nanobots Problem", pulp.LpMaximize)
  x = pulp.LpVariable('x', cat='Integer')
  y = pulp.LpVariable('y', cat='Integer')
  z = pulp.LpVariable('z', cat='Integer')
  counts = [pulp.LpVariable('c_%d' % i, lowBound=0, upBound=1, cat='Integer') for i in range(len(inputs))]
  
  # Main objective
  lp_problem += sum(counts), "Z"
  
  # Add constraints (note that 0 <= c <= 1 constraints are already included)
  for i, (xi, yi, zi, ri) in enumerate(inputs):
    lp_problem += (x - xi) + (y - yi) + (z - zi) <= ri + K * (1 - counts[i])
    lp_problem += (x - xi) + (y - yi) - (z - zi) <= ri + K * (1 - counts[i])
    lp_problem += (x - xi) - (y - yi) + (z - zi) <= ri + K * (1 - counts[i])
    lp_problem += (x - xi) - (y - yi) - (z - zi) <= ri + K * (1 - counts[i])
    lp_problem += - (x - xi) + (y - yi) + (z - zi) <= ri + K * (1 - counts[i])
    lp_problem += - (x - xi) + (y - yi) - (z - zi) <= ri + K * (1 - counts[i])
    lp_problem += - (x - xi) - (y - yi) + (z - zi) <= ri + K * (1 - counts[i])
    lp_problem += - (x - xi) - (y - yi) - (z - zi) <= ri + K * (1 - counts[i])
    
  # Solve
  lp_problem.solve()
  print('Solved status:', pulp.LpStatus[lp_problem.status])
  print('Optimal point: (%d, %d, %d)' % (x.varValue, y.varValue, z.varValue))
  print('Distance to origin: %d' % 
        (np.abs(x.varValue) + np.abs(y.varValue) + np.abs(z.varValue)))
  return lp_problem
  
_ = get_widest_in_range(inputs, K=1000000000)

Solved status: Optimal
Optimal point: (23700389, 42553861, 58368752)
Distance to origin: 124623002
